# 6조 (17011709 정선아, 17011741 문성용, 17011742 김소영)

In [173]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [174]:
MovieLens_df = pd.read_table("ratings.dat", sep='::', header=None, names=['UserID','MovieID','Rating','Timestamp'])
MovieLens_df.drop(['Timestamp'], axis=1, inplace=True)
MovieLens_pivot = MovieLens_df.pivot_table(index='UserID', columns='MovieID', values='Rating')
row_index = MovieLens_pivot.isnull().sum(axis = 1).sort_values() # NaN이 적은 순으로 행 뽑아내기
MovieLens_pivot = pd.DataFrame(MovieLens_pivot, index=row_index.index)
# rating이 많은 사용자 1,000명
MovieLens_pivot = MovieLens_pivot.iloc[:1000,:]
col_index = MovieLens_pivot.isnull().sum(axis = 0).sort_values() # NaN이 적은 순으로 열 뽑아내기
MovieLens_pivot = pd.DataFrame(MovieLens_pivot, columns=col_index.index)
# rating이 많은 영화 1,000개
MovieLens_pivot = MovieLens_pivot.iloc[:,:1000]
# user_index = MovieLens_pivot.index.values # User ID 순서 기억하기
# user_index = pd.Series(user_index) # Index rename Simi인자로 넣기 위해 형태 변경
MovieLens_pivot.fillna(0, inplace=True)
MovieLens_pivot = MovieLens_pivot.astype('int64')
MovieLens_pivot = np.array(MovieLens_pivot) # 함수 인자로 넣기 위해 numpy array 형태로 변경

# 1. Cosine Similarity

In [202]:
def COS(data):
    num_users = np.size(data, axis=0)
    simCOS = np.zeros((num_users,num_users))  # 0으로 초기화 된 행렬 생성
    for u in range(0, num_users):  # 각 유저별로 for문 반복
        arridx_u = np.where(data[u, ] == 0)
        for v in range(u + 1, num_users):
            arridx_v = np.where(data[v, ] == 0)
            arridx = np.unique(np.concatenate((arridx_u, arridx_v), axis=None))

            U = np.delete(data[u, ], arridx)
            V = np.delete(data[v, ], arridx)

            if np.linalg.norm(U) == 0 or np.linalg.norm(V) == 0:
                simCOS[u, v] = 0
            else:
                simCOS[u, v] = np.dot(U, V) / (np.linalg.norm(U) * np.linalg.norm(V))

            simCOS[v, u] = simCOS[u, v]
    return simCOS

# 2. Pearson Correlation Coefficient

In [224]:
def PCC(data):
    num_users = np.size(data, axis=0)
    simPCC = np.zeros((num_users,num_users))  # 0으로 초기화 된 행렬 생성
    mean = np.nanmean(np.where(data != 0, data, np.nan), axis=1)
    for u in range(0, num_users):
        arridx_u = np.where(data[u, ] == 0)
        for v in range(u + 1, num_users):
            arridx_v = np.where(data[v, ] == 0)
            arridx = np.unique(np.concatenate((arridx_u, arridx_v), axis=None))

            U = np.delete(data[u, ], arridx) - mean[u]
            V = np.delete(data[v, ], arridx) - mean[v]

            if np.linalg.norm(U) == 0 or np.linalg.norm(V) == 0:
                simPCC[u, v] = 0
            else:
                simPCC[u, v] = np.dot(U, V) / (np.linalg.norm(U) * np.linalg.norm(V))

            simPCC[v, u] = simPCC[u, v]

    return simPCC

# Rating

In [303]:
def basic_baseline(data, sim, k):
    # initializing (1000, 1000) numpy array with zeros
    predicted_rating = np.zeros(data.shape)

    # calculating means
    mean = np.nanmean(np.where(data != 0, data, np.nan))  # the mean of all ratings
    mean_u = np.nanmean(np.where(data != 0, data, np.nan), axis=1)  # the mean of all users
    mean_i = np.nanmean(np.where(data != 0, data, np.nan), axis=0)  # the mean of all items

    # base user, item
    b_u = mean_u - mean
    b_i = mean_i - mean
    # b_ui = mean + b_u + b_i

    # selecting similarity fuction
    if sim == 'COS':
        sim = COS(data)
    elif sim == 'PCC':
        sim = PCC(data)
        
    # selecting top k users by sorting similarity array
    k_users = np.argsort(-sim)
    k_users = np.delete(k_users, np.s_[k:], 1)  # delete colomn under k

    # number of users with axis = 0 condition
    num_users = np.size(data, axis=0)
    num_items = np.size(data, axis=1)

    
    # calculating predicted ratings
    for u in range(0, num_users):
        list_sim = sim[u, k_users[u]]  # selecting top k similarity
        for i in range(0, num_items):
            list_rating = data[k_users[u], i].astype('float64')  # k users' ratings on item i

            b_ui = mean + b_u[u] + b_i[i]  # scalar
            b_vi = mean + b_u[k_users[u]] + b_i[i]  # list

            # explanation of varialbles
            # mean_u[u] : user u의 평균
            # mean_i[i] : item i의 평균
            # b_u[u] : user u의 baseline
            # b_i[i] : item i의 baseline

            # calculation
            mom = np.sum(list_sim)  # 분모
            son = np.sum(list_sim * (list_rating - b_vi))  # 분자
            predicted_rating[u, i] = b_ui + son / mom

    return predicted_rating